In [1]:
import nltk

In [2]:

import numpy as np
import pandas as pd
import nltk
import tensorflow

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer    # transfor the over data
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer


from tensorflow.keras.utils import pad_sequences  ## 



from tensorflow.keras.layers import LSTM,Dense , SimpleRNN , Embedding , Dropout



In [3]:
train_data = open("data/train.txt").readlines()
test_data = open("data/test.txt").readlines()
val_data = open("data/val.txt").readlines()

In [4]:
len(train_data)

16000

In [5]:
len(test_data)

2000

In [6]:
len(val_data)

2000

In [7]:
total_message = train_data+test_data+val_data

In [8]:
total_message[10]

'i feel like i have to make the suffering i m seeing mean something;sadness\n'

In [9]:
x=[]
y=[]
message_list=[]
message_list

[]

In [10]:
for item in total_message:
    text,label = item.split(';')
    label = label.replace('\n', '')
    x.append(text)
    y.append(label)

In [11]:
label_count = {}
for label in y:
    if label in label_count:
        label_count[label] +=1
    else:
        label_count[label] =1
print('number of uniqque', label_count)

for label, count in label_count.items():
    print(f"{label}:{count}")

number of uniqque {'sadness': 5797, 'anger': 2709, 'love': 1641, 'surprise': 719, 'fear': 2373, 'joy': 6761}
sadness:5797
anger:2709
love:1641
surprise:719
fear:2373
joy:6761


In [12]:
set(y)

{'anger', 'fear', 'joy', 'love', 'sadness', 'surprise'}

In [13]:
len(set(y))

6

In [14]:
df = pd.DataFrame(y)
df.value_counts()

0       
joy         6761
sadness     5797
anger       2709
fear        2373
love        1641
surprise     719
Name: count, dtype: int64

In [15]:
y[:5]

['sadness', 'sadness', 'anger', 'love', 'anger']

In [16]:
x[:5]

['i didnt feel humiliated',
 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
 'im grabbing a minute to post i feel greedy wrong',
 'i am ever feeling nostalgic about the fireplace i will know that it is still on the property',
 'i am feeling grouchy']

In [69]:
stemming = PorterStemmer()
def text_cleaning(ls_sentences):
    cleaned_message_ls = []
    for sentence in ls_sentences:
        lower_sentence = sentence.lower()  #lower case
        ls_of_words = nltk.word_tokenize(lower_sentence)
        ls_of_words_removed_stopwords = cleaned = [i for i in ls_of_words if i not in stopwords.words('english')] #removing stopwords

        stemmed_words_ls = [stemming.stem(word) for word in ls_of_words_removed_stopwords] #stemming apllied
        cleaned_message = " ".join(stemmed_words_ls)  #joining every words with an  space to make a sentence
        cleaned_message_ls.append(cleaned_message)  #adding all words to the list
    return cleaned_message_ls

cleaned_message_ls = text_cleaning(x)

In [18]:
x[30]

'i get giddy over feeling elegant in a perfectly fitted pencil skirt'

In [19]:
cleaned_message_ls[30]

'get giddi feel eleg perfectli fit pencil skirt'

In [20]:

tokenizer= Tokenizer(oov_token="<nothing>")

In [21]:
tokenizer.fit_on_texts(cleaned_message_ls)

In [22]:
tokenizer.document_count

20000

In [23]:
# tokenizer.word_counts

In [24]:
tokenizer.word_index

{'<nothing>': 1,
 'feel': 2,
 'like': 3,
 'im': 4,
 'get': 5,
 'time': 6,
 'know': 7,
 'realli': 8,
 'make': 9,
 'go': 10,
 'want': 11,
 'love': 12,
 'littl': 13,
 'think': 14,
 'peopl': 15,
 'day': 16,
 'thing': 17,
 'one': 18,
 'would': 19,
 'even': 20,
 'still': 21,
 'ive': 22,
 'life': 23,
 'bit': 24,
 'way': 25,
 'need': 26,
 'someth': 27,
 'much': 28,
 'dont': 29,
 'work': 30,
 'start': 31,
 'could': 32,
 'say': 33,
 'look': 34,
 'see': 35,
 'tri': 36,
 'back': 37,
 'good': 38,
 'pretti': 39,
 'come': 40,
 'right': 41,
 'alway': 42,
 'help': 43,
 'also': 44,
 'today': 45,
 'year': 46,
 'take': 47,
 'friend': 48,
 'use': 49,
 'around': 50,
 'cant': 51,
 'person': 52,
 'made': 53,
 'though': 54,
 'hate': 55,
 'well': 56,
 'got': 57,
 'happi': 58,
 'thought': 59,
 'someon': 60,
 'didnt': 61,
 'never': 62,
 'felt': 63,
 'find': 64,
 'write': 65,
 'lot': 66,
 'hope': 67,
 'quit': 68,
 'live': 69,
 'week': 70,
 'everi': 71,
 'sure': 72,
 'less': 73,
 'read': 74,
 'enough': 75,
 'give':

In [25]:
sequence = tokenizer.texts_to_sequences(cleaned_message_ls)

In [26]:
cleaned_message_ls[0]

'didnt feel humili'

In [27]:
sequence[0]

[61, 2, 522]

In [28]:
max(list(map(len,sequence)))

35

In [29]:
sequences = pad_sequences(sequences=sequence,padding='post',maxlen=35)

In [30]:
sequences

array([[  61,    2,  522, ...,    0,    0,    0],
       [  10,    2,  419, ...,    0,    0,    0],
       [   4, 1230,  431, ...,    0,    0,    0],
       ...,
       [   2,  194,  157, ...,    0,    0,    0],
       [ 328,    2,  175, ...,    0,    0,    0],
       [   2,    3,  916, ...,    0,    0,    0]])

In [31]:
# nltk.download('punkt')

In [32]:
y

['sadness',
 'sadness',
 'anger',
 'love',
 'anger',
 'sadness',
 'surprise',
 'fear',
 'joy',
 'love',
 'sadness',
 'joy',
 'anger',
 'sadness',
 'joy',
 'joy',
 'sadness',
 'sadness',
 'sadness',
 'fear',
 'anger',
 'fear',
 'joy',
 'joy',
 'anger',
 'sadness',
 'sadness',
 'sadness',
 'anger',
 'joy',
 'joy',
 'fear',
 'surprise',
 'anger',
 'joy',
 'joy',
 'joy',
 'joy',
 'anger',
 'joy',
 'joy',
 'joy',
 'joy',
 'joy',
 'sadness',
 'sadness',
 'joy',
 'love',
 'joy',
 'anger',
 'joy',
 'sadness',
 'anger',
 'fear',
 'joy',
 'sadness',
 'sadness',
 'surprise',
 'joy',
 'joy',
 'joy',
 'love',
 'fear',
 'fear',
 'surprise',
 'anger',
 'anger',
 'sadness',
 'love',
 'joy',
 'sadness',
 'sadness',
 'joy',
 'sadness',
 'sadness',
 'sadness',
 'joy',
 'joy',
 'joy',
 'anger',
 'sadness',
 'anger',
 'anger',
 'anger',
 'joy',
 'joy',
 'joy',
 'joy',
 'sadness',
 'fear',
 'love',
 'anger',
 'sadness',
 'anger',
 'love',
 'sadness',
 'joy',
 'joy',
 'sadness',
 'anger',
 'love',
 'joy',
 '

In [33]:
label_dict = {
  'sadness':0,
  'anger':1,
  'love':2,
  'surprise':3,
  'fear':4,
  'joy':5
}

def label_encoder(labels):
  label=[]
  for lab in labels:
    label.append(label_dict[lab])
  label = np.array(label)
  return label
  
label = label_encoder(y)


In [34]:
label

array([0, 0, 1, ..., 5, 5, 5])

In [35]:
x_train,x_test,y_train,y_test = train_test_split(sequences,label,test_size=0.2)

In [36]:
x_train,x_test,y_train,y_test

(array([[   2, 1512,  273, ...,    0,    0,    0],
        [ 908,  238,    8, ...,    0,    0,    0],
        [   7,   15,  133, ...,    0,    0,    0],
        ...,
        [ 429,   16,  601, ...,    0,    0,    0],
        [   2,  385,  286, ...,    0,    0,    0],
        [ 187,  949,   19, ...,    0,    0,    0]]),
 array([[   2,   24,  592, ...,    0,    0,    0],
        [ 168, 1876,  697, ...,    0,    0,    0],
        [   5,   38,    2, ...,    0,    0,    0],
        ...,
        [ 254,  254,   43, ...,    0,    0,    0],
        [1389,    9,    2, ...,    0,    0,    0],
        [ 410, 1690,  187, ...,    0,    0,    0]]),
 array([5, 3, 1, ..., 0, 5, 0]),
 array([2, 5, 5, ..., 0, 0, 5]))

In [37]:
x_train.shape,y_train.shape

((16000, 35), (16000,))

In [38]:
x_test.shape , y_test.shape

((4000, 35), (4000,))

In [39]:
### model train 
# ml   
# ANN  -------->>>>>  ml algo related like classification regression 
# CNN ----->>>  when we have grapical data like image video 
# RNN ---->>>text data , sequntial data



In [40]:
model = Sequential()
model.add(SimpleRNN(32,input_shape=(35,1),return_sequences=False)) 

model.add(Dense(6,activation='softmax'))# multi layer

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

C:\Users\dalpa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [41]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 32)             │         1,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 6)              │           198 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,286 (5.02 KB)

 Trainable params: 1,286 (5.02 KB)

 Non-trainable params: 0 (0.00 B)

In [42]:
history = model.fit(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.3289 - loss: 1.6055 - val_accuracy: 0.3422 - val_loss: 1.5746
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.3313 - loss: 1.5851 - val_accuracy: 0.3428 - val_loss: 1.5736
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.3294 - loss: 1.5851 - val_accuracy: 0.3415 - val_loss: 1.5724
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.3374 - loss: 1.5777 - val_accuracy: 0.3422 - val_loss: 1.5735
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.3383 - loss: 1.5784 - val_accuracy: 0.3405 - val_loss: 1.5748


In [43]:
pd.Series(y).value_counts()

joy         6761
sadness     5797
anger       2709
fear        2373
love        1641
surprise     719
Name: count, dtype: int64

In [49]:
######### using lstm
# model2 = Sequential()
# model2.add()

model2 = Sequential([
  LSTM(units=72,input_shape=(35,1),return_sequences=True),
  LSTM(units=64,dropout=0.4),
  Dense(units=6,activation='softmax')
])
model2.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

C:\Users\dalpa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [50]:
model2.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 35, 72)         │        21,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 64)             │        35,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 6)              │           390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 56,774 (221.77 KB)

 Trainable params: 56,774 (221.77 KB)

 Non-trainable params: 0 (0.00 B)

In [51]:
history = model2.fit(x_train,y_train,epochs=20,validation_data=(x_test,y_test))

Epoch 1/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.3239 - loss: 1.5983 - val_accuracy: 0.3422 - val_loss: 1.5782
Epoch 2/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.3419 - loss: 1.5661 - val_accuracy: 0.3422 - val_loss: 1.5737
Epoch 3/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.3395 - loss: 1.5735 - val_accuracy: 0.3072 - val_loss: 1.5792
Epoch 4/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.3293 - loss: 1.5840 - val_accuracy: 0.3450 - val_loss: 1.5727
Epoch 5/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.3324 - loss: 1.5717 - val_accuracy: 0.3422 - val_loss: 1.5710
Epoch 6/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.3344 - loss: 1.5759 - val_accuracy: 0.3510 - val_loss: 1.5703
Epoch 7/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 11s 21ms/step - accuracy: 0.3341 - loss: 1.5701 - val_accuracy: 0.3462 - val_loss: 1.5715
Epoch 8/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.3444 - loss: 1.5674 - val_

In [52]:
import joblib

In [56]:
model2.save('LSTM_model.h5')

In [61]:
prediction = model2.predict(x_test)


125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step


In [62]:
x_test.shape

(4000, 35)

In [64]:
prediction.shape

(4000, 6)

In [65]:
y_test[1]

5

In [66]:
prediction[1]

array([0.27757788, 0.16948411, 0.08539884, 0.04749981, 0.12611096,
       0.29392847], dtype=float32)

In [67]:
np.argmax(prediction[1])

5

In [58]:
from tensorflow.keras.models import load_model

In [59]:
a = load_model('LSTM_model.h5')

In [60]:
a

<Sequential name=sequential_2, built=True>